In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import urllib.request
import pandas as pd
import scipy.sparse as sp
import os

In [7]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [8]:
g = dataset[0]

In [9]:
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={})

In [10]:
# Positive edges
u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())

In [11]:
train_pos_g

Graph(num_nodes=2708, num_edges=9501,
      ndata_schemes={}
      edata_schemes={})

In [12]:
test_pos_g

Graph(num_nodes=2708, num_edges=1055,
      ndata_schemes={}
      edata_schemes={})

In [13]:

neg_u1 = np.repeat(g.nodes(),g.out_degrees(g.nodes()))[::]
neg_v1 = np.random.choice(g.num_nodes(), len(neg_u1))
train_neg_g = dgl.graph((neg_u1, neg_v1), num_nodes=g.number_of_nodes())

#neg_u1 = np.repeat(g.nodes(),1)[::2]
neg_u1 = np.random.choice(g.num_nodes(), test_size)
neg_v1 = np.random.choice(g.num_nodes(), len(neg_u1))
test_neg_g = dgl.graph((neg_u1, neg_v1), num_nodes=g.number_of_nodes())

In [14]:
train_neg_g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={}
      edata_schemes={})

In [15]:
test_neg_g

Graph(num_nodes=2708, num_edges=1055,
      ndata_schemes={}
      edata_schemes={})

In [16]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [17]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h
    

In [18]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [19]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [20]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [21]:
model

GraphSAGE(
  (conv1): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_self): Linear(in_features=1433, out_features=16, bias=False)
    (fc_neigh): Linear(in_features=1433, out_features=16, bias=False)
  )
  (conv2): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_self): Linear(in_features=16, out_features=16, bias=False)
    (fc_neigh): Linear(in_features=16, out_features=16, bias=False)
  )
)

In [24]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.1)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(1000):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 25 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))


# Thumbnail credits: Link Prediction with Neo4j, Mark Needham
# sphinx_gallery_thumbnail_path = '_static/blitz_4_link_predict.png'

/Users/sganguly/opt/anaconda3/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 0.42548638582229614
In epoch 25, loss: 0.5310391783714294
In epoch 50, loss: 0.4571548104286194
In epoch 75, loss: 0.44469067454338074
In epoch 100, loss: 0.4400877356529236
In epoch 125, loss: 0.43810033798217773
In epoch 150, loss: 0.4366475045681
In epoch 175, loss: 0.4355567991733551
In epoch 200, loss: 0.4345090091228485
In epoch 225, loss: 0.4327739179134369
In epoch 250, loss: 0.43211451172828674
In epoch 275, loss: 0.43260321021080017
In epoch 300, loss: 0.43006631731987
In epoch 325, loss: 0.4287421405315399
In epoch 350, loss: 0.42793476581573486
In epoch 375, loss: 0.42706504464149475
In epoch 400, loss: 0.4278598129749298
In epoch 425, loss: 0.42663881182670593
In epoch 450, loss: 0.42597338557243347
In epoch 475, loss: 0.4299991726875305
In epoch 500, loss: 0.4256109297275543
In epoch 525, loss: 0.43197891116142273
In epoch 550, loss: 0.43211135268211365
In epoch 575, loss: 0.4251275360584259
In epoch 600, loss: 0.4266597032546997
In epoch 625, loss: 0.42